# Lesson 4: Creating a cell counting module in PyCharm

Today we will be building a module in Python from scratch. The goal is to have this module do a little bit of histology analysis. It won't produce publication-ready figures, but can at least show you the beginnings of what you can accomplish using Python. 

# Step -1: Conceptualize the problem

The first step before writing any code at all is to think about the algorithm behind the intended analysis. At technical interviews for software engineering positions, interviewers ask for code that will solve a particular problem that they present. Before you even touch the keyboard, you must first think about the problem and the steps that you need to code up to solve the problem. Then you communicate your thought process to the interviewer. It is imperative that you understand what the problem is before you try to solve it. 

Today we will write some code to plot some confocal images with circled cell bodies. 

**What are the steps required to accomplish this? List them in order.**

# Step 0: Create new Python module

Before you start coding, you will need to create a .py file to open in the PyCharm editor. 

1. Open PyCharm and your PyCharm project. Your project should automatically load when you start up PyCharm, but if not, go to File -> Open -> navigate to PythonForNeuro folder -> Ok.
2. On the menu to the left, right click PythonForNeuro folder -> New -> Python File -> Type in an appropriate name (e.g., histology) -> Ok. 

The typical anatomy of a Python module is as follows in this specific order: 

1. Import relevant outside packages (e.g., `numpy`).
2. All the code that you develop (e.g., functions). 
3. A `__name__` block to make debugging easier. 

Step 3 may not make sense right now, but we will get to it soon. Most of my code will use `numpy` and `matplotlib.pyplot`. `import` those as `np` and `plt`.

# Step 1: Load .czi file

The first thing we want to do is to load the contents of the .czi file. At the beinning of your Python module, _make sure to import the necessary package(s)_. Then, define a function that takes an input `fname`, the name of the .czi file. Name it `load_frame`. 

In the body of the function definition, copy and paste the following: 

In [ ]:
with CziFile(fname) as czi:
    im = czi.asarray()

This code block will open the .czi file, temporarily store its contents in `czi`, then convert that convert that content into an array. Next, enter code that will return the array and exit the function. 

# Step 1.1: Debug load function

Cool, now how can we check whether this works? This is where debugging comes in. Debugging is very useful for examining checkpoints and checking whether code is doing what you think it's doing. That requires going inside the processes and functions that you've defined and inspecting variables to see whether they match your expectations. Let's set up the `__name__` block.

A generic `__name__` block looks like this: 

In [ ]:
if __name__ == '__main__':
    #CODE HERE

For now, copy and paste this: 

In [ ]:
if __name__ == '__main__':
    fname = #file name here
    
    frame = load_frame(fname)

The details behind why this is necessary are not critical, but if you are curious, read this: https://stackoverflow.com/questions/419163/what-does-if-name-main-do. Basically, whenever you debug the file, Python will run everything above the `__name__` block (which will be `import` statements and function definitions). Then the code in the `__name__` block will call those functions (e.g., `load_frame`). 

To debug, first insert a _breakpoint_ in `load_frame` by clicking the space in between the line number and the actual code. Insert this breakpoint on the line that returns your frame in the `load_frame` function. A red dot should appear on that line. Now, when you enter debug mode, the `__name__` block will run and the code will stop before the breakpoint. Try doing this by right clicking on the editor and clicking `Debug 'histology'`. 

You should see the line with the breakpoint highlighted. 
1. Click on the Debugger tab and you will see the variables in the workspace. 
2. Inspect `im`. What is its data type? What are its dimensions? Is this expected? 
3. You can click `View as Array` to look at its values. 


# Step 1.2: Cleaning and smoothing data

For this step, we will:
1. Get rid of the singleton dimensions in `im`. 
2. Extract a specific frame from `im`. 
3. Smooth that frame.

At the same time, we will explore the debugging environment (note that I'm using the word environment to mean something different from a conda env). 

## Step 1.2.1: Resolving singleton dimensions

`im` has a lot of singleton dimensions which don't contain any data. 
1. Use the `numpy` function `squeeze()` to remove these dimensions and add this code to your `load_frame` definition. 
2. Place a breakpoint on this line, remove the previous breakpoint, then enter the debugger again (you can click the green circular arrow to the left of the Debugger tab). 
3. You are now stopped right before `squeeze()` happens. Inspect `im`. 
4. Then click the bent blue arrow (`Step Over`). This causes Python to run the next line (which should be the `squeeze` call) and stop. 
5. Inspect the output of your `squeeze`. How is it different from the original `im`? 

## Step 1.2.2: Extract subset of the data
Our squeezed `im` should have the shape `(3, 13, 1946, 1946)`. This corresponds to (channel, z position, x position, y position) in our array of z stacks. Let's define our `load_frame` function to also select a channel and a z position. Add these as arguments into your definition. Enter a line of code that takes a frame from a channel and a z position specified by the input arguments. I recommend testing on `z=10` and `channel=1`. 

Click the calculator-like button (`Evaluate Expression`). This opens up a window that lets you enter code inside the debugging environment. Use this to plot the frame with `plt.imshow()`. 

## Step 1.2.3: Smooth image

Now, smooth your image. Use the function `gaussian_filter` from `scipy.ndimage.filters`. Refer to this reference: https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.ndimage.filters.gaussian_filter.html. I recommend using a `sigma` of 3 while keeping the other parameters default. 

At any point in this process, restart the debugger and insert breakpoints whenever you want to inspect variables and see whether your function is performing as intended. Change stuff and see what happens. At the end of this step, you should return a `(1946, 1946)` array. 

# Step 2: Detect blobs (cells)

Now we can look for our cells. Luckily, blob detection code already exists. 

1. Define a new function that calls `blob_log` from the `skimage.feature` package and name it `detect_blobs`. It should return the output of `blob_log`. 
2. In the `__name__` block, add a call to `detect_blobs()`. 
3. `detect_blobs` should take parameters `threshold`, `min_sigma`, and `max_sigma` that will be passed into `blob_log`. For defaults, I recommend `threshold=0.04`, `min_sigma=10`, `max_sigma=30`. Refer to this documentation for more information: https://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.blob_log.

# Step 3: Plot

Next we will plot our detected cells on top of the confocal image. First we must plot our confocal image. 

## Step 3.1: Plot confocal image

Define a function called `plot_blobs` that takes as inputs the outputs of `load_frame` and `detect_blobs`. As the first line of `plot_blobs`, enter:

In [ ]:
fig, ax = plt.subplots()

`ax` will be a variable storing an `Axes` object on which you can plot things. This will be useful for when we want to overlay multiple data types. 

Insert a line of code that will plot the confocal image using `ax.imshow()`. `imshow()` is a method of the `ax` object that plots an image. Store the output of `ax.imshow()` in a variable `i`. Inspect `i` in the debugging environment and you will see that it's an image that resides in a plotted axis. 

It will be helpful for visualization purposes if we manually manipulate the color limits of the plot. Insert the following line after `ax.imshow()`:

In [ ]:
i.set_clim(15, im.max()/2)

## Step 3.2: Plot blobs

Next, plot your blobs. Using a `for` loop, loop through all your detected blobs and plot them. Refer to this for an example: https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_blob.html. 

## Step 3.3: Color

Once you have that working, add some color options to your confocal image plotting tools. Add `channel` as an argument to `plot_blobs`. Use an `if` statement to define conditions when the confocal image color palette should be either `'Blues'` or `'Greens'`. This can be implemented as an argument in `ax.imshow()` as `cmap=image_color`.

Within the same `if` statements, define conditions when the blobs should be circled in a different color, `'red'` versus `'black'`. This can be implemented as an argument in `plt.Circle()` as `color=circle_color`.

# Step 4: Wrapper function

Finally, define a function called `analyze_frame` that runs all the steps in order for both c-fos and DAPI channels. This is our "wrapper" function. It should take two arguments, `fname` and `z`. 
1. Move `fig, ax = plt.subplots()` out of `plot_blobs` and into `analyze_frame`. Now we can use the `ax` object to plot both channels' images as well as both of their detected cells. 
2. Write two lines to load frames from the same z plane but from different channels. 
3. Detect blobs from those two channels. 
4. Plot the images and cells from those two channels. I recommend plotting the c-fos channel first because the DAPI signal tends to be smothered if that gets plotted first. You will also have to include a new argument into `ax.imshow()` and `plt.Circle()` -- set `alpha=0.7` for both function calls. 

# Step 5: Run it!

You're done! Now you can run it in the Python Console. 

In [ ]:
import histology
fname = # file name here

for i in range(14):
    histology.analyze_frame(fname, z=i)